#Books Recommender system usin clustering | Collaborative based

In [2]:
import pandas as pd
import numpy as np
import matplotlib as plt
import seaborn as sns

In [9]:
books = pd.read_csv("BX-Books.csv",sep=";", on_bad_lines='skip', encoding="latin-1") # skip bad lines
books.head(2)

<ipython-input-9-f1ba14c6d49e>:1: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  books = pd.read_csv("BX-Books.csv",sep=";", on_bad_lines='skip', encoding="latin-1") # skip bad lines


,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L
0,0195153448,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...
1,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...


In [10]:
books.shape

(271360, 8)

In [11]:
books.columns

Index(['ISBN', 'Book-Title', 'Book-Author', 'Year-Of-Publication', 'Publisher',
       'Image-URL-S', 'Image-URL-M', 'Image-URL-L'],
      dtype='object')

In [12]:
books=books[["ISBN","Book-Title","Book-Author","Year-Of-Publication", "Publisher","Image-URL-L"]]

In [13]:
books.head(2)

,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-L
0,0195153448,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press,http://images.amazon.com/images/P/0195153448.0...
1,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...


In [14]:
books.rename(columns={"Book-Title":"title",
                      "Book-Author":"author",
                      "Year-Of-Publication":"year",
                      "Publisher":"publisher",
                      "Image-URL-L":"image_url"}, inplace=True)

In [15]:
books.head(2)

,ISBN,title,author,year,publisher,image_url
0,0195153448,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press,http://images.amazon.com/images/P/0195153448.0...
1,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...


In [16]:
users = pd.read_csv("BX-Users.csv",sep=";", on_bad_lines='skip', encoding="latin-1") # skip bad lines
users.head(1)

,User-ID,Location,Age
0,1,"nyc, new york, usa",NaN


In [17]:
users.shape

(278858, 3)

In [18]:
ratings = pd.read_csv("BX-Book-Ratings.csv",sep=";", on_bad_lines='skip', encoding="latin-1") # skip bad lines
ratings.head(2)

,User-ID,ISBN,Book-Rating
0,276725,034545104X,0
1,276726,0155061224,5


In [19]:
ratings.shape

(1149780, 3)

In [20]:
print(books.shape)
print(users.shape)
print(ratings.shape)

(271360, 6)
(278858, 3)
(1149780, 3)


In [24]:
ratings.rename(columns={
    "User-ID":"user_id",
    "Book-Rating":"rating"
    },inplace=True)

In [26]:
ratings.head(1)

,user_id,ISBN,rating
0,276725,034545104X,0


In [27]:
ratings["user_id"].value_counts()

,count
user_id,
11676,13602
198711,7550
153662,6109
98391,5891
35859,5850
...,...
116180,1
116166,1
116154,1


In [28]:
ratings["user_id"].unique().shape

(105283,)

In [29]:
x = ratings["user_id"].value_counts()>200

In [30]:
x

,count
user_id,
11676,True
198711,True
153662,True
98391,True
35859,True
...,...
116180,False
116166,False
116154,False


In [31]:
x[x].shape

(899,)

In [32]:
y = x[x].index

In [33]:
y

Index([ 11676, 198711, 153662,  98391,  35859, 212898, 278418,  76352, 110973,
       235105,
       ...
       260183,  73681,  44296, 155916,   9856, 274808,  28634,  59727, 268622,
       188951],
      dtype='int64', name='user_id', length=899)

In [36]:
ratings = ratings[ratings["user_id"].isin(y)]

In [37]:
ratings.head()

,user_id,ISBN,rating
1456,277427,002542730X,10
1457,277427,0026217457,0
1458,277427,003008685X,8
1459,277427,0030615321,0
1460,277427,0060002050,0


In [39]:
ratings.shape

(526356, 3)

In [40]:
ratings_with_books = ratings.merge(books,on = "ISBN")

In [41]:
ratings_with_books

,user_id,ISBN,rating,title,author,year,publisher,image_url
0,277427,002542730X,10,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner,1994,John Wiley &amp; Sons Inc,http://images.amazon.com/images/P/002542730X.0...
1,3363,002542730X,0,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner,1994,John Wiley &amp; Sons Inc,http://images.amazon.com/images/P/002542730X.0...
2,11676,002542730X,6,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner,1994,John Wiley &amp; Sons Inc,http://images.amazon.com/images/P/002542730X.0...
3,12538,002542730X,10,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner,1994,John Wiley &amp; Sons Inc,http://images.amazon.com/images/P/002542730X.0...
4,13552,002542730X,0,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner,1994,John Wiley &amp; Sons Inc,http://images.amazon.com/images/P/002542730X.0...
...,...,...,...,...,...,...,...,...
487666,275970,1892145022,0,Here Is New York,E. B. White,1999,Little Bookroom,http://images.amazon.com/images/P/1892145022.0...
487667,275970,1931868123,0,There's a Porcupine in My Outhouse: Misadventu...,Mike Tougias,2002,Capital Books (VA),http://images.amazon.com/images/P/1931868123.0...
487668,275970,3411086211,10,Die Biene.,Sybil GrÃ?Â¤fin SchÃ?Â¶nfeldt,1993,"Bibliographisches Institut, Mannheim",http://images.amazon.com/images/P/3411086211.0...
487669,275970,3829021860,0,The Penis Book,Joseph Cohen,1999,Konemann,http://images.amazon.com/images/P/3829021860.0...


In [42]:
ratings_with_books.shape

(487671, 8)

In [43]:
num_rating = ratings_with_books.groupby("title")["rating"].count().reset_index()

In [44]:
num_rating.head()

,title,rating
0,A Light in the Storm: The Civil War Diary of ...,2
1,Always Have Popsicles,1
2,Apple Magic (The Collector's series),1
3,Beyond IBM: Leadership Marketing and Finance ...,1
4,Clifford Visita El Hospital (Clifford El Gran...,1


In [45]:
num_rating.rename(columns={"rating":"num_of_rating"},inplace=True)

In [46]:
num_rating.head(2)

,title,num_of_rating
0,A Light in the Storm: The Civil War Diary of ...,2
1,Always Have Popsicles,1


In [47]:
final_ratings = ratings_with_books.merge(num_rating,on="title")

In [48]:
final_ratings.head()

,user_id,ISBN,rating,title,author,year,publisher,image_url,num_of_rating
0,277427,002542730X,10,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner,1994,John Wiley &amp; Sons Inc,http://images.amazon.com/images/P/002542730X.0...,82
1,3363,002542730X,0,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner,1994,John Wiley &amp; Sons Inc,http://images.amazon.com/images/P/002542730X.0...,82
2,11676,002542730X,6,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner,1994,John Wiley &amp; Sons Inc,http://images.amazon.com/images/P/002542730X.0...,82
3,12538,002542730X,10,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner,1994,John Wiley &amp; Sons Inc,http://images.amazon.com/images/P/002542730X.0...,82
4,13552,002542730X,0,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner,1994,John Wiley &amp; Sons Inc,http://images.amazon.com/images/P/002542730X.0...,82


In [50]:
final_ratings.shape

(487671, 9)

In [51]:
final_rating = final_ratings[final_ratings["num_of_rating"]>=50]

In [52]:
final_rating.head()

,user_id,ISBN,rating,title,author,year,publisher,image_url,num_of_rating
0,277427,002542730X,10,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner,1994,John Wiley &amp; Sons Inc,http://images.amazon.com/images/P/002542730X.0...,82
1,3363,002542730X,0,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner,1994,John Wiley &amp; Sons Inc,http://images.amazon.com/images/P/002542730X.0...,82
2,11676,002542730X,6,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner,1994,John Wiley &amp; Sons Inc,http://images.amazon.com/images/P/002542730X.0...,82
3,12538,002542730X,10,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner,1994,John Wiley &amp; Sons Inc,http://images.amazon.com/images/P/002542730X.0...,82
4,13552,002542730X,0,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner,1994,John Wiley &amp; Sons Inc,http://images.amazon.com/images/P/002542730X.0...,82


In [54]:
final_rating.sample(10)

,user_id,ISBN,rating,title,author,year,publisher,image_url,num_of_rating
170557,76352,0312926294,0,The Carousel,Rosamunde Pilcher,1991,St. Martin's Press,http://images.amazon.com/images/P/0312926294.0...,56
103200,6563,0312924801,0,September,Rosamunde Pilcher,1991,St. Martin's Press,http://images.amazon.com/images/P/0312924801.0...,50
3690,35859,0380710218,0,Postmortem,Patricia Daniels Cornwell,1993,Harper Mass Market Paperbacks (Mm),http://images.amazon.com/images/P/0380710218.0...,78
14761,188951,0345351525,0,The Queen of the Damned (Vampire Chronicles (P...,Anne Rice,1993,Ballantine Books,http://images.amazon.com/images/P/0345351525.0...,96
118353,127233,031231616X,0,I Capture the Castle,Dodie Smith,2003,St. Martin's Griffin,http://images.amazon.com/images/P/031231616X.0...,66
88015,8067,0312980140,10,Seven Up (A Stephanie Plum Novel),Janet Evanovich,2002,St. Martin's Paperbacks,http://images.amazon.com/images/P/0312980140.0...,111
15952,238120,0446607657,0,When the Wind Blows,James Patterson,1999,Warner Vision,http://images.amazon.com/images/P/0446607657.0...,175
34277,225810,0440206154,0,Red Dragon,Thomas Harris,2000,Dell Publishing Company,http://images.amazon.com/images/P/0440206154.0...,151
127417,255489,0446610542,0,The Bachelor (Warner Books Contemporary Romance),Carly Phillips,2002,Warner Forever,http://images.amazon.com/images/P/0446610542.0...,71
133051,201768,0425178765,0,Easy Prey,John Sandford,2001,Berkley Publishing Group,http://images.amazon.com/images/P/0425178765.0...,73


In [55]:
final_rating.shape

(61853, 9)

In [56]:
final_rating.drop_duplicates(["user_id","title"],inplace=True)

<ipython-input-56-f1353b39d3da>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_rating.drop_duplicates(["user_id","title"],inplace=True)


In [57]:
final_rating.shape

(59850, 9)

In [58]:
book_pivot = final_rating.pivot_table(columns="user_id",index="title",values= "rating")

In [59]:
book_pivot

user_id,254,2276,2766,2977,3363,3757,4017,4385,6242,6251,...,274004,274061,274301,274308,274808,275970,277427,277478,277639,278418
title,,,,,,,,,,,,,,,,,,,,,
1984,9.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN
1st to Die: A Novel,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2nd Chance,NaN,10.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,0.0,NaN
4 Blondes,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
84 Charing Cross Road,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,10.0,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Year of Wonders,NaN,NaN,NaN,7.0,NaN,NaN,NaN,NaN,7.0,NaN,...,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN
You Belong To Me,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Zen and the Art of Motorcycle Maintenance: An Inquiry into Values,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,0.0,...,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN


In [60]:
book_pivot.shape

(742, 888)

In [61]:
book_pivot.fillna(0,inplace=True)

In [62]:
book_pivot

user_id,254,2276,2766,2977,3363,3757,4017,4385,6242,6251,...,274004,274061,274301,274308,274808,275970,277427,277478,277639,278418
title,,,,,,,,,,,,,,,,,,,,,
1984,9.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1st to Die: A Novel,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2nd Chance,0.0,10.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4 Blondes,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
84 Charing Cross Road,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,10.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Year of Wonders,0.0,0.0,0.0,7.0,0.0,0.0,0.0,0.0,7.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
You Belong To Me,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Zen and the Art of Motorcycle Maintenance: An Inquiry into Values,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [63]:
from scipy.sparse import csr_matrix

In [64]:
book_sparse = csr_matrix(book_pivot)

In [65]:
book_sparse

<742x888 sparse matrix of type '<class 'numpy.float64'>'
	with 14942 stored elements in Compressed Sparse Row format>

In [66]:
from sklearn.neighbors import NearestNeighbors
model = NearestNeighbors(algorithm="brute")

In [67]:
model.fit(book_sparse)

NearestNeighbors(algorithm='brute')

In [68]:
distance, suggestion = model.kneighbors(book_pivot.iloc[237,:].values.reshape(1,-1),n_neighbors=6)

In [69]:
distance

array([[ 0.        , 68.78953409, 69.5413546 , 72.64296249, 76.83098333,
        77.28518616]])

In [70]:
suggestion

array([[237, 240, 238, 241, 184, 536]])

In [71]:
for i in range(len(suggestion)):
  print(book_pivot.index[suggestion[i]])

Index(['Harry Potter and the Chamber of Secrets (Book 2)',
       'Harry Potter and the Prisoner of Azkaban (Book 3)',
       'Harry Potter and the Goblet of Fire (Book 4)',
       'Harry Potter and the Sorcerer's Stone (Book 1)', 'Exclusive',
       'The Cradle Will Fall'],
      dtype='object', name='title')


In [84]:
book_pivot.index[237]

'Harry Potter and the Chamber of Secrets (Book 2)'

In [73]:
books_name = book_pivot.index

In [74]:
books_name

Index(['1984', '1st to Die: A Novel', '2nd Chance', '4 Blondes',
       '84 Charing Cross Road', 'A Bend in the Road', 'A Case of Need',
       'A Child Called \It\": One Child's Courage to Survive"',
       'A Civil Action', 'A Cry In The Night',
       ...
       'Winter Solstice', 'Wish You Well', 'Without Remorse',
       'Wizard and Glass (The Dark Tower, Book 4)', 'Wuthering Heights',
       'Year of Wonders', 'You Belong To Me',
       'Zen and the Art of Motorcycle Maintenance: An Inquiry into Values',
       'Zoya', '\O\" Is for Outlaw"'],
      dtype='object', name='title', length=742)

In [75]:
import pickle
pickle.dump(model, open("artifacts/model.pkl","wb"))
pickle.dump(books_name, open("artifacts/books_name.pkl","wb"))
pickle.dump(final_rating, open("artifacts/final_rating.pkl","wb"))
pickle.dump(book_pivot, open("artifacts/book_pivot.pkl","wb"))


In [76]:
!zip -r /content/artifacts.zip /content/artifacts
from google.colab import files
files.download("/content/artifacts.zip")

  adding: content/artifacts/ (stored 0%)
  adding: content/artifacts/books_name.pkl (deflated 54%)
  adding: content/artifacts/model.pkl (deflated 80%)
  adding: content/artifacts/book_pivot.pkl (deflated 99%)
  adding: content/artifacts/final_rating.pkl (deflated 91%)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [78]:
def recommend_book(book_name):
  book_id = np.where(book_pivot.index==book_name)[0][0]
  distance, suggestion = model.kneighbors(book_pivot.iloc[book_id,:].values.reshape(1,-1),n_neighbors=6)

  for i in range(len(suggestion)):
    books = book_pivot.index[suggestion[i]]
    for j in books:
      print(j)


In [85]:
book_name = 'Harry Potter and the Chamber of Secrets (Book 2)'
recommend_book(book_name)

Harry Potter and the Chamber of Secrets (Book 2)
Harry Potter and the Prisoner of Azkaban (Book 3)
Harry Potter and the Goblet of Fire (Book 4)
Harry Potter and the Sorcerer's Stone (Book 1)
Exclusive
The Cradle Will Fall
